# Season-wise Potential Utilised Through Dugwells
# Relation between the three datasets
So, basically there are three datasets with almost similar names matching the above heading.  
Let's just dig deep into these for a start.

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../Utils/Index/')
import Clean
df1 = Clean.clean(pd.read_csv(
    '../../Datasets/Season wise Potential Utilised (in use) Through Dugwells for 5th Minor Irrigation Census (2013-14).csv'))
df2 = Clean.clean(pd.read_csv(
    '../../Datasets/Season wise Potential Utilised (Temporary not in use) Through Dugwells for 5th Minor Irrigation Census (2013-14).csv'))
df3 = Clean.clean(pd.read_csv(
    '../../Datasets/Season wise Potential Utilised Through All Dugwells for 5th Minor Irrigation Census (2013-14).csv'))

df1 - In use  
df2 - Temporary not in use  
df3 - All

In [2]:
[pd.Series(df1.columns == df2.columns).unique(),
 pd.Series(df2.columns == df3.columns).unique(),
 pd.Series(df3.columns == df1.columns).unique()]

[array([ True]), array([ True]), array([ True])]

unique() returns a list of unique elements from the given series.  
Making sure that the three datasets do have the same attributes across.

In [3]:
df1.shape

(213392, 8)

In [4]:
df2.shape

(34138, 8)

In [5]:
df3.shape

(222992, 8)

Doesn't seem like these are just down to simple arithmetics as the names suggests.   
Let's check the smaller ones with the largest.

In [6]:
df1.shape[0] + df2.shape[0]

247530

`In use` + `not in use` do not give us `all`. (Misconception basically)

In [7]:
pd.Series(df2.index).apply(lambda x : x in df3.index).value_counts()

True    34138
Name: State_District_Block/Tehsil_Village, dtype: int64

That leaves us with the fact that all instances in df2 are present in df3.  
If you're wondering how it made sense, compare the number obtained with the length of df2.  
Let's check if stuff in df1 is present in df3 which in case does will make sense.  

In [8]:
pd.Series(df1.index).apply(lambda x : x in df3.index).value_counts()

True    213392
Name: State_District_Block/Tehsil_Village, dtype: int64

That's another YES saying all data points present in df1 are present in df3.  
Let's check if stuff in df2 is present in df1 which in case does will make sense.  
I expect it won't and the number of false will be the difference between the first and last datasets.

In [9]:
pd.Series(df2.index).apply(lambda x : x in df1.index).value_counts()

True     24538
False     9600
Name: State_District_Block/Tehsil_Village, dtype: int64

As expected, the number of falses is same as the difference between the first and last datasets.  
All this leaves us with some facts about the three datasets.  
df1 contains the areas in each location that's being irrigated currently using dugwells.  
df2 the same but areas that have been temporarily not in use.  
df3 contains literally what we get if we summate the other two datasets.  

In [10]:
li = list()
def add(x, y):
    if x.name in y.index:
        x += y.loc[x.name]
        li.append(x.name)
    return x
df1 = df1.apply(add, y = df2, axis = 1)
df2 = df2.drop(li)

The above block of code adds all the common elements from df1 and df2 and adds them to df1, removes them from df2.

In [11]:
df1.shape[0] + df2.shape[0] == df3.shape[0]

True

In [12]:
df3.equals(df1.append(df2))

False

In [13]:
df4 = df1.append(df2)

df4 just created is expected to be equal to df3.

In [14]:
x = (df3.sort_index() == df4.sort_index())

In [15]:
x.apply(lambda x : x.value_counts())

,Area irrigated during 2013-2014 - Inside Command of Major/Medium Project - Kharif,Area irrigated during 2013-2014 - Inside Command of Major/Medium Project - Rabi,Area irrigated during 2013-2014 - Inside Command of Major/Medium Project - Perennial,Area irrigated during 2013-2014 - Inside Command of Major/Medium Project - Others,Area irrigated during 2013-2014 - Outside Command - Kharif,Area irrigated during 2013-2014 - Outside Command - Rabi,Area irrigated during 2013-2014 - Outside Command - Perennial,Area irrigated during 2013-2014 - Outside Command - Others
True,222926,222935,222967,222990,221503,221705,222677,222802
False,66,57,25,2,1489,1287,315,190


In [16]:
for i in range(8):
    print(str(df3.sum()[i]) + '\n' + str(df4.sum()[i]))

332108.180000002
332108.1800000023
363455.99999999994
363456.0000000001
83788.62999999987
83788.62999999987
14739.070000000036
14739.070000000036
6112357.040000006
6112357.039999994
8721889.879999977
8721889.879999964
783704.0699999997
783704.0699999998
435031.4500000022
435031.45000000135


In my quest for getting df3 from df1 and df2 brings me down to some negligible round-off errors.
# That's a conclusion to stop this relation analysis